In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import *
#from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.neural_network import MLPClassifier

### Data Cleaning Begins

    """
    Until where indicated, all code taken from

    Long, A. (2020, January 30).
        Using Machine Learning to Predict Hospital Readmission for Patients with
        Diabetes with Scikit-Learn. Retrieved November 21, 2020,
        from https://towardsdatascience.com/predicting-hospital-readmission-for-
        patients-with-diabetes-using-scikit-learn-a2e359b15f0
        
    Except for some comments, which I've added
    """
    
Except for some comments, which I've added
"""
"""
Until where indicated, all code taken from
https://github.com/vignesh-bhat1999/predicting_hospital_readmissions
Vignesh B Svignesh-bhat1999
Machine learning and deep learning enthusiast
All notes are preserved for better understanding of the Data Cleaning method

In [5]:
def dataPrep(fileName):

    """
    Until where indicated, all code taken from

    Long, A. (2020, January 30).
        Using Machine Learning to Predict Hospital Readmission for Patients with
        Diabetes with Scikit-Learn. Retrieved November 21, 2020,
        from https://towardsdatascience.com/predicting-hospital-readmission-for-
        patients-with-diabetes-using-scikit-learn-a2e359b15f0

    Except for some comments, which I've added
    """

    #DATA EXPLORATION
    
    # load the csv file
    df = pd.read_csv(fileName)
    #print('Number of samples:',len(df)) #flag
    #gets the columns
    #df.info()

    # count the number of rows for each type
    df.groupby('readmitted').size() #gives number of readmissions
    df.groupby('discharge_disposition_id').size()
    
    #removing rows of patients who died or went to hospice
    df = df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

    #output variable to make binary classifications with
    #will predict whether a patient will be readmitted within 30 days of their
    #discharge from the hospital
    df['OUTPUT_LABEL'] = (df.readmitted == '<30').astype('int')

    #print('Prevalence:%.3f'%calc_prevalence(df['OUTPUT_LABEL'].values)) #flag

    #FEATURE ENGINEERING
    
    df[list(df.columns)[:10]].head()
    df[list(df.columns)[10:20]].head()
    df[list(df.columns)[20:30]].head()
    df[list(df.columns)[30:40]].head()
    df[list(df.columns)[40:]].head()

    # for each column
    for c in list(df.columns):

        # get a list of unique values
        n = df[c].unique()

        #flags
        """
        # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
        if len(n)<30:
            print(c)
            print(n)
        else:
            print(c + ': ' +str(len(n)) + ' unique values')
        """

    
    # replace ? with nan
    df = df.replace('?',np.nan)

    #NUMERICAL FEATURES
    cols_num = ['time_in_hospital','num_lab_procedures', 'num_procedures',
                'num_medications', 'number_outpatient', 'number_emergency',
                'number_inpatient','number_diagnoses']

    df[cols_num].isnull().sum()

    #CATEGORICAL FEATURES
    cols_cat = ['race', 'gender',
           'max_glu_serum', 'A1Cresult',
           'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
           'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
           'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
           'miglitol', 'troglitazone', 'tolazamide', 'insulin',
           'glyburide-metformin', 'glipizide-metformin',
           'glimepiride-pioglitazone', 'metformin-rosiglitazone',
           'metformin-pioglitazone', 'change', 'diabetesMed','payer_code']

    df[cols_cat].isnull().sum()

    #replacing Na/NaN values with "UNK"
    df['race'] = df['race'].fillna('UNK')
    df['payer_code'] = df['payer_code'].fillna('UNK')
    df['medical_specialty'] = df['medical_specialty'].fillna('UNK')

    #print('Number medical specialty:', df.medical_specialty.nunique()) #flag
    df.groupby('medical_specialty').size().sort_values(ascending = False)

    #Top 10 features from medical specialty
    top_10 = ['UNK','InternalMedicine','Emergency/Trauma',\
              'Family/GeneralPractice', 'Cardiology','Surgery-General' ,\
              'Nephrology','Orthopedics',\
              'Orthopedics-Reconstructive','Radiologist']

    # make a new column with duplicated data
    df['med_spec'] = df['medical_specialty'].copy()

    # replace all specialties not in top 10 with 'Other' category
    df.loc[~df.med_spec.isin(top_10),'med_spec'] = 'Other'

    df.groupby('med_spec').size()
    cols_cat_num = ['admission_type_id', 'discharge_disposition_id',
                    'admission_source_id']
    df[cols_cat_num] = df[cols_cat_num].astype('str')
    df_cat = pd.get_dummies(df[cols_cat + cols_cat_num + ['med_spec']],
                            drop_first = True)
    df_cat.head()
    df = pd.concat([df,df_cat], axis = 1)
    cols_all_cat = list(df_cat.columns)

    #EXTRA FEATURES
    df[['age', 'weight']].head()
    df.groupby('age').size()
    #reformatting age_id from non-numeric to numeric
    age_id = {'[0-10)':0,
              '[10-20)':10,
              '[20-30)':20,
              '[30-40)':30,
              '[40-50)':40,
              '[50-60)':50,
              '[60-70)':60,
              '[70-80)':70,
              '[80-90)':80,
              '[90-100)':90}
    df['age_group'] = df.age.replace(age_id)
    df.weight.notnull().sum()
    df['has_weight'] = df.weight.notnull().astype('int')
    cols_extra = ['age_group','has_weight']

    #flags
    """
    print('Total number of features:', len(cols_num + cols_all_cat + cols_extra))
    print('Numerical Features:',len(cols_num))
    print('Categorical Features:',len(cols_all_cat))
    print('Extra features:',len(cols_extra))
    """

    df[cols_num + cols_all_cat + cols_extra].isnull().sum().sort_values(ascending = False).head(10)
    col2use = cols_num + cols_all_cat + cols_extra
    df_data = df[col2use + ['OUTPUT_LABEL']]

    # shuffle the samples
    df_data = df_data.sample(n = len(df_data), random_state = 42)
    df_data = df_data.reset_index(drop = True)

    # # Save 30% of the data as validation and test data
    # df_valid_test=df_data.sample(frac=0.30,random_state=42)
    # #print('Split size: %.3f'%(len(df_valid_test)/len(df_data))) #flag
    # 
    # df_test = df_valid_test.sample(frac = 0.5, random_state = 42)
    # df_valid = df_valid_test.drop(df_test.index)
    # 
    # # use the rest of the data as training data
    # df_train_all=df_data.drop(df_valid_test.index)
    
    #flags
    """
    print('Test prevalence(n = %d):%.3f'%(len(df_test),calc_prevalence(df_test.OUTPUT_LABEL.values)))
    print('Valid prevalence(n = %d):%.3f'%(len(df_valid),calc_prevalence(df_valid.OUTPUT_LABEL.values)))
    print('Train all prevalence(n = %d):%.3f'%(len(df_train_all), calc_prevalence(df_train_all.OUTPUT_LABEL.values)))
    """
    
    #print('all samples (n = %d)'%len(df_data)) #flag
    assert len(df_data) == (len(df_data)),'math didnt work'

    # split the training data into positive and negative
    rows_pos = df_data.OUTPUT_LABEL == 1
    df_train_pos = df_data.loc[rows_pos]
    df_train_neg = df_data.loc[~rows_pos]

    # merge the balanced data
    df_train = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 42)],axis = 0)

    # shuffle the order of training samples
    df_train = df_train.sample(n = len(df_train), random_state = 42).reset_index(drop = True)

    """
    print('Train balanced prevalence(n = %d):%.3f'%(len(df_train), \
                    calc_prevalence(df_train.OUTPUT_LABEL.values))) #flag
    """
    
    # df_train_all.to_csv('df_train_all.csv',index=False)
    df_train.to_csv('df_train.csv',index=False)
    # df_valid.to_csv('df_valid.csv',index=False)
    # df_test.to_csv('df_test.csv',index=False)

    X_ = df_train[col2use].values
    # X_train_all = df_train_all[col2use].values
    # X_valid = df_valid[col2use].values

    y_ = df_train['OUTPUT_LABEL'].values
    # y_valid = df_valid['OUTPUT_LABEL'].values

    #flags
    """
    print('Training All shapes:',X_train_all.shape)
    print('Training shapes:',X_train.shape, y_train.shape)
    print('Validation shapes:',X_valid.shape, y_valid.shape)
    """

    scaler  = StandardScaler()
    scaler.fit(X_)
    scalerfile = 'scaler.sav'
    pickle.dump(scaler, open(scalerfile, 'wb'))
    # load it back
    scaler = pickle.load(open(scalerfile, 'rb'))

    X_ = scaler.transform(X_)
    # X_valid_tf = scaler.transform(X_valid)

    return X_, y_

In [6]:
training_file = 'C2T1_Train.csv'
testing_file = 'C2T1_Test.csv'


In [7]:
X_train, y_train = dataPrep(training_file)
X_test, y_test = dataPrep(training_file)

### Data Preparation Done!!!

### MLP Model

In [8]:
class Perceptron:
    def __init__(self, learningRate=0.01, nIterations=1000):
        self.learningRate = learningRate
        self.nIterations = nIterations
        self.activationFunc = self._stepFunc
        self.weights = None
        self.bias = None

    #goes through the training data x and attempts to fit it to y
    def fit(self, x,y):
        # num of rows is the number of samples
        # num of cols is the number of features
        nSamples,nFeatures = x.shape

        #setting weights to 0 for every feature
        self.weights = np.zeros(nFeatures)
        self.bias = 0

        for _ in range(self.nIterations):
            #enumerate function gives index and current sample
            for idx, x_i in enumerate(x):

                #gets prediction
                yPredicted = self.predict(x_i)

                #checks which way we need to adjust the weights
                if (y[idx] > yPredicted):
                    update = self.learningRate
                elif(y[idx] < yPredicted):
                    update = -self.learningRate
                #good so dont change
                else:
                    update = 0

                self.weights += update * x_i
                self.bias += update

    #used to predict current input with current weights + bias
    def predict(self, x):
        #takes the dot product
        #w transpose * X + bias
        linearOuput = np.dot(x, self.weights) + self.bias
        #apply activation function
        y_predicted = self.activationFunc(linearOuput)
        return y_predicted


    #activation function
    def _stepFunc(self, x):
        #allows x to be a single sample or x to be an array
        #threshold
        return np.where(x>=0, 1, 0)

#perceptron + the addition of pocketalgo
class PocketAlgo:
    def __init__(self, learningRate=0.01, nIterations=1000):
        self.learningRate = learningRate
        self.nIterations = nIterations
        self.activationFunc = self._stepFunc
        self.longestRun = 0
        self.weights = None
        self.pocket = None
        self.bias = None

    #goes through the training data x and attempts to fit it to y
    def fit(self, x,y):
        # num of rows is the number of samples
        # num of cols is the number of features
        nSamples,nFeatures = x.shape

        #initialize weights
        #setting weights to 0 for every feature
        self.weights = np.zeros(nFeatures)
        self.pocket =  np.zeros(nFeatures)
        self.bias = 0


        for _ in range(self.nIterations):

            currentLongestRun = 0
            bestCurrentWeights = self.pocket.copy()
            self.weights = self.pocket.copy()
            #enumerate function gives index and current sample
            for idx, x_i in enumerate(x):
                currentLongestRun += 1
                yPredicted = self.predict(x_i)

                #checks if the current prediction is incorrect
                #if so checks if a new weight vector needs to be replaced in the pocket

                if(y[idx] != yPredicted):
                    if(y[idx] > yPredicted):
                        update = self.learningRate
                    elif(y[idx] < yPredicted):
                        update = -self.learningRate
                    if (currentLongestRun > self.longestRun):
                        bestCurrentWeights = self.weights.copy()
                        self.longestRun = currentLongestRun
                    currentLongestRun = 0

                    self.weights += update * x_i
                    self.bias += update


            #sets the bestCurrentWeights to pocket- may not change the value of the pocket
            self.pocket = bestCurrentWeights.copy()


    def predict(self, x):
        #takes the dot product
        #w transpose * X + bias
        linearOuput = np.dot(x, self.weights) + self.bias
        #apply activation function
        y_predicted = self.activationFunc(linearOuput)
        return y_predicted


    #activation function
    def _stepFunc(self, x):
        #allows x to be a single sample or x to be an array
        #threshold
        return np.where(x>=0, 1, 0)


### Training of the MMLP Begins

In [9]:
classifyReadmissionsPerception = PocketAlgo(learningRate=0.001, nIterations=1000)
classifyReadmissionsPerception.fit(X_train, y_train) # fitting

y_train_preds = classifyReadmissionsPerception.predict(X_train)
y_test_preds = classifyReadmissionsPerception.predict(X_test)

### Specificity Report

In [10]:
thresh =0.5

def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh):
    auc = roc_auc_score(y_actual, y_pred)
    print(y_pred)
    print((y_pred >thresh))
    print(y_actual)
    #y_actual list of [1,0,0,0,1..]
    #y_pred list of decimals 0-1
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

In [11]:
def printConfusionMatrix(accuracy,precision,recall,specificity,auc,y_train,str_title=""):
    print("--------------------"+str_title+"Confusion Matrix Finishied----------------------------------")
    print("The accuracy is:", accuracy)
#     print("PRECISION for NOT_Readmiteed_in_30days:",TN/(TN+FN))
#     print("RECALL        for NOT_Readmiteed_in_30days:",TN/(TN+FP))
#     print("SPECIFICITY   for NOT_Readmiteed_in_30days:",TP/(TN+FP))    
#         print("Accuracy  for NOT_Readmiteed_in_30days:",(TP + TN)/(TP+TN+FP+FN))
    print("PRECISION for   Readmiteed_in_30days:",precision)
    print("RECALL    for   Readmiteed_in_30days:",recall)
    print("SPECIFICITY for Readmiteed_in_30days:",specificity)
#         print("Accuracy for      Readmiteed_in_30days:",(TP + TN)/(TP+TN+FP+FN))
    print("ROC_AUC=",auc)
    print("Prevalence=",calc_prevalence(y_train))
    print(50*"--")

    return 

In [14]:
print("Perception")
print('On training data')
auc, accuracy, recall, precision, specificity = print_report(y_train,y_train_preds, thresh)
printConfusionMatrix(accuracy,precision,recall,specificity,auc,y_train,str_title="")
print('\n\n\nTest:')
auc, accuracy, recall, precision, specificity = print_report(y_test,y_test_preds, thresh)
printConfusionMatrix(accuracy,precision,recall,specificity,auc,y_test,str_title="")

Perception
On training data
[0.50275566 0.53983799 0.53714795 ... 0.64114694 0.68115906 0.60485204]
[ True  True  True ...  True  True  True]
[1 1 1 ... 1 1 0]
AUC:0.667
accuracy:0.615
recall:0.548
precision:0.633
specificity:0.682
prevalence:0.500
 
--------------------Confusion Matrix Finishied----------------------------------
The accuracy is: 0.6149328859060402
PRECISION for   Readmiteed_in_30days: 0.6327368061096547
RECALL    for   Readmiteed_in_30days: 0.5478681405448085
SPECIFICITY for Readmiteed_in_30days: 0.681997631267272
ROC_AUC= 0.6666220360253731
Prevalence= 0.5
----------------------------------------------------------------------------------------------------



Test:
[0.50275566 0.53983799 0.53714795 ... 0.64114694 0.68115906 0.60485204]
[ True  True  True ...  True  True  True]
[1 1 1 ... 1 1 0]
AUC:0.667
accuracy:0.615
recall:0.548
precision:0.633
specificity:0.682
prevalence:0.500
 
--------------------Confusion Matrix Finishied----------------------------------
The 

### Comparison with Library

In [15]:
ann = MLPClassifier(learning_rate='constant',learning_rate_init=0.001, max_iter= 10000, activation='logistic',solver='sgd',hidden_layer_sizes=(35),random_state=1)
ann.fit(X_train,y_train)


y_train_preds = ann.predict_proba(X_train)[:,1]
y_test_preds = ann.predict_proba(X_test)[:,1]

auc, accuracy, recall, precision, specificity = print_report(y_train,y_train_preds, thresh)
printConfusionMatrix(accuracy,precision,recall,specificity,auc,y_train,str_title="")
print('\n\n\nTest:')
auc, accuracy, recall, precision, specificity = print_report(y_test,y_test_preds, thresh)
printConfusionMatrix(accuracy,precision,recall,specificity,auc,y_test,str_title="")


[0.50275566 0.53983799 0.53714795 ... 0.64114694 0.68115906 0.60485204]
[ True  True  True ...  True  True  True]
[1 1 1 ... 1 1 0]
AUC:0.667
accuracy:0.615
recall:0.548
precision:0.633
specificity:0.682
prevalence:0.500
 
--------------------Confusion Matrix Finishied----------------------------------
The accuracy is: 0.6149328859060402
PRECISION for   Readmiteed_in_30days: 0.6327368061096547
RECALL    for   Readmiteed_in_30days: 0.5478681405448085
SPECIFICITY for Readmiteed_in_30days: 0.681997631267272
ROC_AUC= 0.6666220360253731
Prevalence= 0.5
----------------------------------------------------------------------------------------------------



Test:
[0.50275566 0.53983799 0.53714795 ... 0.64114694 0.68115906 0.60485204]
[ True  True  True ...  True  True  True]
[1 1 1 ... 1 1 0]
AUC:0.667
accuracy:0.615
recall:0.548
precision:0.633
specificity:0.682
prevalence:0.500
 
--------------------Confusion Matrix Finishied----------------------------------
The accuracy is: 0.6149328859060